In [1]:
import keras
import numpy as np
from keras.applications import inception_v3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K

from preprocess import *
from datagen import DataGenerator

K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [ ]:
inception_model = inception_v3.InceptionV3(weights='imagenet', include_top = False)

In [ ]:
x = inception_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 38 classes
predictions = Dense(38, activation='softmax')(x)

In [ ]:
model = Model(inputs = inception_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [3]:
#ONLY NEED TO RUN THIS CELL ONCE EVER (although nothing will happen if it is run more than once)!!!!!!

#Now for the data:
#This function loops through the class folders created by CrowdAI, and renames the images
#copies them to trainingdump and validationdump folders in the same directory

loop_through_and_rename()

In [ ]:
#Create a list of training and validation features in the form of image file names, which will be used by the data generator
training_features_list = create_features_list("data/crowdai_train/crowdai/trainingdump")
validation_features_list = create_features_list("data/crowdai_train/crowdai/validationdump")

In [ ]:
training_labels_list = create_labels_list(training_features_list)
validation_labels_list = create_labels_list(validation_features_list)

In [ ]:
params = {'dim': (299, 299),
          'batch_size': 32,
          'n_classes': 38,
          'n_channels': 3,
          'shuffle': True}

In [ ]:
training_generator = DataGenerator(training_features_list, training_labels_list, **params)
validation_generator = DataGenerator(validation_features_list, validation_labels_list, **params)

model.fit_generator(generator=training_generator, validation_data=validation_generator, use_multiprocessing=True, workers=6)#, use_multiprocessing=True, workers=6, verbose=1)